<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/city_list_wikidata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sparqlwrapper

     |████████████████████████████████| 348kB 22.1MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 


In [2]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=db4b9b076e788f50a1c5109b7ae68efe83f645df60e45cc54a168aca245fec63
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import pandas as pd
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import re
from collections import Counter
import time
import itertools
import wikipedia
import requests
from pandas.io.json import json_normalize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
      item = []
      for c in cols:
        item.append(row.get(c, {}).get('value'))
      out.append(item)
    return pd.DataFrame(out, columns = cols)

In [0]:

url = "https://query.wikidata.org/sparql"

query = '''
SELECT ?instance_of ?instance_ofLabel ?country ?countryLabel ?located_in_the_administrative_territorial_entity ?located_in_the_administrative_territorial_entityLabel WHERE 
{
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?instance_of wdt:P31 wd:Q515.
  OPTIONAL { ?instance_of wdt:P17 ?country. }
  OPTIONAL { ?instance_of wdt:P131 ?located_in_the_administrative_territorial_entity. }
}
'''
#  print(query)

df = get_results(url, query)

In [7]:
df.head()

,instance_of,instance_ofLabel,country,countryLabel,located_in_the_administrative_territorial_entity,located_in_the_administrative_territorial_entityLabel
0,http://www.wikidata.org/entity/Q3940,Victoria,http://www.wikidata.org/entity/Q1042,Seychelles,http://www.wikidata.org/entity/Q1042,Seychelles
1,http://www.wikidata.org/entity/Q3992,Liège,http://www.wikidata.org/entity/Q31,Belgium,http://www.wikidata.org/entity/Q1127,Liège
2,http://www.wikidata.org/entity/Q4035,Osasco,http://www.wikidata.org/entity/Q155,Brazil,http://www.wikidata.org/entity/Q175,São Paulo
3,http://www.wikidata.org/entity/Q4045,Fujairah,http://www.wikidata.org/entity/Q878,United Arab Emirates,http://www.wikidata.org/entity/Q4091,Fujairah
4,http://www.wikidata.org/entity/Q4093,Glasgow,http://www.wikidata.org/entity/Q145,United Kingdom,http://www.wikidata.org/entity/Q55934339,Glasgow City


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10084 entries, 0 to 10083
Data columns (total 6 columns):
instance_of                                              10084 non-null object
instance_ofLabel                                         10084 non-null object
country                                                  9605 non-null object
countryLabel                                             9605 non-null object
located_in_the_administrative_territorial_entity         8465 non-null object
located_in_the_administrative_territorial_entityLabel    8465 non-null object
dtypes: object(6)
memory usage: 472.8+ KB


In [9]:
df.sort_values(['countryLabel','located_in_the_administrative_territorial_entityLabel','instance_ofLabel'],inplace=True)

,instance_of,instance_ofLabel,country,countryLabel,located_in_the_administrative_territorial_entity,located_in_the_administrative_territorial_entityLabel
9359,http://www.wikidata.org/entity/Q11929668,Karkh,http://www.wikidata.org/entity/Q12536,Abbasid Caliphate,None,None
9318,http://www.wikidata.org/entity/Q11925148,Q11925148,http://www.wikidata.org/entity/Q26857,Aetolia,None,None
6568,http://www.wikidata.org/entity/Q685808,Asadabad,http://www.wikidata.org/entity/Q889,Afghanistan,http://www.wikidata.org/entity/Q2663257,Asadabad
6639,http://www.wikidata.org/entity/Q1020649,Samangan,http://www.wikidata.org/entity/Q889,Afghanistan,http://www.wikidata.org/entity/Q3694404,Aybak
7049,http://www.wikidata.org/entity/Q732879,Baghlan,http://www.wikidata.org/entity/Q889,Afghanistan,http://www.wikidata.org/entity/Q170309,Baghlan
...,...,...,...,...,...,...
10015,http://www.wikidata.org/entity/Q65088955,Triopium,None,None,None,None
7960,http://www.wikidata.org/entity/Q5373099,Ur Kaśdim,None,None,None,None
8680,http://www.wikidata.org/entity/Q46187190,Uscana,None,None,None,None
9263,http://www.wikidata.org/entity/Q11878638,Velká Olomouc,None,None,None,None


In [6]:
df = df[['countryLabel','located_in_the_administrative_territorial_entityLabel','instance_ofLabel',\
         'country','located_in_the_administrative_territorial_entity','instance_of']]
df.head()

,countryLabel,located_in_the_administrative_territorial_entityLabel,instance_ofLabel,country,located_in_the_administrative_territorial_entity,instance_of
0,Seychelles,Seychelles,Victoria,http://www.wikidata.org/entity/Q1042,http://www.wikidata.org/entity/Q1042,http://www.wikidata.org/entity/Q3940
1,Belgium,Liège,Liège,http://www.wikidata.org/entity/Q31,http://www.wikidata.org/entity/Q1127,http://www.wikidata.org/entity/Q3992
2,Brazil,São Paulo,Osasco,http://www.wikidata.org/entity/Q155,http://www.wikidata.org/entity/Q175,http://www.wikidata.org/entity/Q4035
3,United Arab Emirates,Fujairah,Fujairah,http://www.wikidata.org/entity/Q878,http://www.wikidata.org/entity/Q4091,http://www.wikidata.org/entity/Q4045
4,United Kingdom,Glasgow City,Glasgow,http://www.wikidata.org/entity/Q145,http://www.wikidata.org/entity/Q55934339,http://www.wikidata.org/entity/Q4093


In [0]:
df.sort_values(['countryLabel','located_in_the_administrative_territorial_entityLabel','instance_ofLabel'],inplace=True)

In [12]:
df.head()

,countryLabel,located_in_the_administrative_territorial_entityLabel,instance_ofLabel,country,located_in_the_administrative_territorial_entity,instance_of
9359,Abbasid Caliphate,None,Karkh,http://www.wikidata.org/entity/Q12536,None,http://www.wikidata.org/entity/Q11929668
9318,Aetolia,None,Q11925148,http://www.wikidata.org/entity/Q26857,None,http://www.wikidata.org/entity/Q11925148
6568,Afghanistan,Asadabad,Asadabad,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q2663257,http://www.wikidata.org/entity/Q685808
6639,Afghanistan,Aybak,Samangan,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q3694404,http://www.wikidata.org/entity/Q1020649
7049,Afghanistan,Baghlan,Baghlan,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q170309,http://www.wikidata.org/entity/Q732879


In [13]:
df.head(10)

,countryLabel,located_in_the_administrative_territorial_entityLabel,instance_ofLabel,country,located_in_the_administrative_territorial_entity,instance_of
9359,Abbasid Caliphate,None,Karkh,http://www.wikidata.org/entity/Q12536,None,http://www.wikidata.org/entity/Q11929668
9318,Aetolia,None,Q11925148,http://www.wikidata.org/entity/Q26857,None,http://www.wikidata.org/entity/Q11925148
6568,Afghanistan,Asadabad,Asadabad,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q2663257,http://www.wikidata.org/entity/Q685808
6639,Afghanistan,Aybak,Samangan,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q3694404,http://www.wikidata.org/entity/Q1020649
7049,Afghanistan,Baghlan,Baghlan,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q170309,http://www.wikidata.org/entity/Q732879
5549,Afghanistan,Balkh,Kholm,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q121104,http://www.wikidata.org/entity/Q1014375
2578,Afghanistan,Balkh,Mazar-i-Sharif,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q121104,http://www.wikidata.org/entity/Q130469
1811,Afghanistan,Bamyan,Bamyan,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q171382,http://www.wikidata.org/entity/Q214495
4796,Afghanistan,Farah,Farah,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q180330,http://www.wikidata.org/entity/Q593156
7208,Afghanistan,Faryab,Andkhoy,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q173830,http://www.wikidata.org/entity/Q1054127


In [14]:
df.shape

(10084, 6)

In [16]:
df = df.reset_index(drop=True)
df.head()

,countryLabel,located_in_the_administrative_territorial_entityLabel,instance_ofLabel,country,located_in_the_administrative_territorial_entity,instance_of
0,Abbasid Caliphate,None,Karkh,http://www.wikidata.org/entity/Q12536,None,http://www.wikidata.org/entity/Q11929668
1,Aetolia,None,Q11925148,http://www.wikidata.org/entity/Q26857,None,http://www.wikidata.org/entity/Q11925148
2,Afghanistan,Asadabad,Asadabad,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q2663257,http://www.wikidata.org/entity/Q685808
3,Afghanistan,Aybak,Samangan,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q3694404,http://www.wikidata.org/entity/Q1020649
4,Afghanistan,Baghlan,Baghlan,http://www.wikidata.org/entity/Q889,http://www.wikidata.org/entity/Q170309,http://www.wikidata.org/entity/Q732879


In [0]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
#    print('Wrote {} records to {}'.format(len(data), output_path))



def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [0]:
def get_wikidata_id(col):
  rx = re.compile(r'(http\:\/\/www\.wikidata\.org\/entity\/)(Q[0-9]+)')
  m = rx.match(col)
  if m is not None:
    return m.group(2)

def get_wikidata_description(row):
  base_string = 'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&languages=en&format=json&props=descriptions&ids='
  query_string = base_string + row
#  print(query_string)
  r = requests.get(query_string)
  return json.loads(r.content.decode('utf-8'))['entities'][row]

def get_wikidata_label(row):
  base_string = 'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&languages=en&format=json&props=labels&ids='
  query_string = base_string + row
#  print(query_string)
  r = requests.get(query_string)
  return json.loads(r.content.decode('utf-8'))['entities'][row]

In [0]:
def query_and_process_results(url, query_parameter):


  query = query_parameter
#  print(query)

  df = get_results(url, query)
  df.shape
  df.head()

  res_list =  sorted([re.sub("q[0-9]+","",item.lower()) for item in df.groupby(['instance_ofLabel'])['instance_of'].agg('count').index.values.tolist()])
  res_list = sorted([re.sub("\-", " ",item) for item in res_list])
  res_list = [item for item in res_list if not re.findall("[0-9]+",item)]
  res_counter = Counter(res_list)
  res_list = []
  for key in res_counter.keys():
    if len(key) != 0:
      res_list.append(key)

  print(len(res_list), res_list)

  return df, res_list, res_counter

In [0]:
url = "https://query.wikidata.org/sparql"

query = '''
SELECT ?instance_of ?instance_ofLabel ?country ?countryLabel ?located_in_the_administrative_territorial_entity ?located_in_the_administrative_territorial_entityLabel WHERE 
{
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?instance_of wdt:P31 wd:Q515.
  OPTIONAL { ?instance_of wdt:P17 ?country. }
  OPTIONAL { ?instance_of wdt:P131 ?located_in_the_administrative_territorial_entity. }
}
'''

In [11]:
df2, r_list, r_counter = query_and_process_results(url, query)

7602 ["'s hertogenbosch", 'a coruña', 'aabenraa', 'aasiaat', 'aba', 'abala', 'abancay', 'abaradira', 'abasan al kabera', 'abbeyleix', 'abbotsford', 'abbottabad', 'abdon', 'abengourou', 'abeokuta', 'abercwmboi', 'aberdeen', 'abha', 'abidjan', 'abkua', 'aboisso', 'abomey', 'abomsa', 'abong mbang', 'abreus', 'abu dhabi', 'abu dis', 'abu kabir', 'abu qir', 'abu qirqas', 'abu qurayn', 'abutig', 'abéché', 'abū qurqāş', 'acalayong', 'acapulco', 'acarigua', 'acarlar', 'accra', 'achacachi', 'achaguas', 'acomayo', 'aconibe', 'acre', 'acuera', 'acultzingo', "ad dali'", 'ad dhahiriya', "ad dhale'e district", 'ad dis', 'ad quintum', 'ad salices', 'adabokrom', 'adamantina', 'adamov', 'adana', 'adapazarı', 'addu city', 'adelaide', 'aden', 'adet', 'adi keyh', 'adi quala', 'adigrat', 'ado ekiti', 'adobha', 'adolfo', 'adolfo ruiz cortines', 'adrar', 'adrogué', 'adré', 'adzopé', 'adıyaman', 'aegys', 'aelia capitolina', 'afak', 'afif', 'aflou', 'afrin', 'afula', 'afyonkarahisar', 'agadir', 'agaete', 'agar

In [12]:
df2.head()

,instance_of,instance_ofLabel,country,countryLabel,located_in_the_administrative_territorial_entity,located_in_the_administrative_territorial_entityLabel
0,http://www.wikidata.org/entity/Q1903,Catania,http://www.wikidata.org/entity/Q38,Italy,http://www.wikidata.org/entity/Q20991246,Metropolitan City of Catania
1,http://www.wikidata.org/entity/Q1906,Caserta,http://www.wikidata.org/entity/Q38,Italy,http://www.wikidata.org/entity/Q16153,Province of Caserta
2,http://www.wikidata.org/entity/Q1947,Juba,http://www.wikidata.org/entity/Q958,South Sudan,http://www.wikidata.org/entity/Q487709,Central Equatoria
3,http://www.wikidata.org/entity/Q1960,Dar es Salaam,http://www.wikidata.org/entity/Q924,Tanzania,http://www.wikidata.org/entity/Q557539,Dar es Salaam Region
4,http://www.wikidata.org/entity/Q1963,Khartoum,http://www.wikidata.org/entity/Q1049,Sudan,http://www.wikidata.org/entity/Q310385,Khartoum


In [13]:
df2.head()

,instance_of,instance_ofLabel,country,countryLabel,located_in_the_administrative_territorial_entity,located_in_the_administrative_territorial_entityLabel
0,http://www.wikidata.org/entity/Q1903,Catania,http://www.wikidata.org/entity/Q38,Italy,http://www.wikidata.org/entity/Q20991246,Metropolitan City of Catania
1,http://www.wikidata.org/entity/Q1906,Caserta,http://www.wikidata.org/entity/Q38,Italy,http://www.wikidata.org/entity/Q16153,Province of Caserta
2,http://www.wikidata.org/entity/Q1947,Juba,http://www.wikidata.org/entity/Q958,South Sudan,http://www.wikidata.org/entity/Q487709,Central Equatoria
3,http://www.wikidata.org/entity/Q1960,Dar es Salaam,http://www.wikidata.org/entity/Q924,Tanzania,http://www.wikidata.org/entity/Q557539,Dar es Salaam Region
4,http://www.wikidata.org/entity/Q1963,Khartoum,http://www.wikidata.org/entity/Q1049,Sudan,http://www.wikidata.org/entity/Q310385,Khartoum


In [15]:
df2['wikidata_entity_id'] = df2['instance_of'].apply(get_wikidata_id)
df2.head()

,instance_of,instance_ofLabel,country,countryLabel,located_in_the_administrative_territorial_entity,located_in_the_administrative_territorial_entityLabel,wikidata_entity_id
0,http://www.wikidata.org/entity/Q1903,Catania,http://www.wikidata.org/entity/Q38,Italy,http://www.wikidata.org/entity/Q20991246,Metropolitan City of Catania,Q1903
1,http://www.wikidata.org/entity/Q1906,Caserta,http://www.wikidata.org/entity/Q38,Italy,http://www.wikidata.org/entity/Q16153,Province of Caserta,Q1906
2,http://www.wikidata.org/entity/Q1947,Juba,http://www.wikidata.org/entity/Q958,South Sudan,http://www.wikidata.org/entity/Q487709,Central Equatoria,Q1947
3,http://www.wikidata.org/entity/Q1960,Dar es Salaam,http://www.wikidata.org/entity/Q924,Tanzania,http://www.wikidata.org/entity/Q557539,Dar es Salaam Region,Q1960
4,http://www.wikidata.org/entity/Q1963,Khartoum,http://www.wikidata.org/entity/Q1049,Sudan,http://www.wikidata.org/entity/Q310385,Khartoum,Q1963


In [0]:
  df2['desc_detail'] = df2['wikidata_entity_id'].apply(get_wikidata_description)
  df2['label_detail'] = df2['wikidata_entity_id'].apply(get_wikidata_label)

In [0]:
df2.to_csv('city_details.csv')

In [0]:
import spacy

In [0]:
nlp = spacy.load('en_core_web_sm')

In [22]:
df2['instance_ofLabel']

0              Catania
1              Caserta
2                 Juba
3        Dar es Salaam
4             Khartoum
             ...      
10082         Kerkrade
10083        Q28976219
10084        Q28976366
10085        Q29000726
10086    Al-Mashariqah
Name: instance_ofLabel, Length: 10087, dtype: object

In [23]:
r_list[:10]

["'s hertogenbosch",
 'a coruña',
 'aabenraa',
 'aasiaat',
 'aba',
 'abala',
 'abancay',
 'abaradira',
 'abasan al kabera',
 'abbeyleix']

In [27]:
[print(token.text) for token in nlp.pipe(r_list[1:2])]

a coruña


[None]

In [28]:
r_list[0]

"'s hertogenbosch"

In [33]:
['{"LOWER": "'+ token.text + '"}' for token in nlp(r_list[0])]

['{"LOWER": "\'s"}', '{"LOWER": "hertogenbosch"}']

In [51]:
patterns = []
id_list = []
for item in r_list:
  patterns.append(['{"LOWER": "'+ token.text + '"}' for token in nlp(item)])
  id_list.append(['_'+ token.text for token in nlp(item)])

print(patterns)
print(id_list)

[['{"LOWER": "\'s"}', '{"LOWER": "hertogenbosch"}'], ['{"LOWER": "a"}', '{"LOWER": "coruña"}'], ['{"LOWER": "aabenraa"}'], ['{"LOWER": "aasiaat"}'], ['{"LOWER": "aba"}'], ['{"LOWER": "abala"}'], ['{"LOWER": "abancay"}'], ['{"LOWER": "abaradira"}'], ['{"LOWER": "abasan"}', '{"LOWER": "al"}', '{"LOWER": "kabera"}'], ['{"LOWER": "abbeyleix"}'], ['{"LOWER": "abbotsford"}'], ['{"LOWER": "abbottabad"}'], ['{"LOWER": "abdon"}'], ['{"LOWER": "abengourou"}'], ['{"LOWER": "abeokuta"}'], ['{"LOWER": "abercwmboi"}'], ['{"LOWER": "aberdeen"}'], ['{"LOWER": "abha"}'], ['{"LOWER": "abidjan"}'], ['{"LOWER": "abkua"}'], ['{"LOWER": "aboisso"}'], ['{"LOWER": "abomey"}'], ['{"LOWER": "abomsa"}'], ['{"LOWER": "abong"}', '{"LOWER": "mbang"}'], ['{"LOWER": "abreus"}'], ['{"LOWER": "abu"}', '{"LOWER": "dhabi"}'], ['{"LOWER": "abu"}', '{"LOWER": "dis"}'], ['{"LOWER": "abu"}', '{"LOWER": "kabir"}'], ['{"LOWER": "abu"}', '{"LOWER": "qir"}'], ['{"LOWER": "abu"}', '{"LOWER": "qirqas"}'], ['{"LOWER": "abu"}', '{"L

In [54]:
print(len(patterns),len(id_list))

7602 7602


In [42]:
with open("pats.txt", "w") as output:
    output.write(str(patterns))

343

In [0]:
df_pats = pd.DataFrame(patterns)
df_ids = pd.DataFrame(id_list)

In [119]:
df_pats['combined'] = df_pats[0].astype('str') + ', ' + df_pats[1].astype('str') + ', ' + df_pats[2].astype('str') + ', ' + df_pats[3].astype('str')\
+ ', ' + df_pats[4].astype('str') + ', ' + df_pats[5].astype('str') + ', ' + df_pats[6].astype('str') + ', ' + df_pats[7].astype('str')
df_pats.drop(columns=[0,1,2,3,4,5,6,7],inplace=True)
df_pats.head()

,combined
0,"{""LOWER"": ""'s""}, {""LOWER"": ""hertogenbosch""}, N..."
1,"{""LOWER"": ""a""}, {""LOWER"": ""coruña""}, None, Non..."
2,"{""LOWER"": ""aabenraa""}, None, None, None, None,..."
3,"{""LOWER"": ""aasiaat""}, None, None, None, None, ..."
4,"{""LOWER"": ""aba""}, None, None, None, None, None..."


In [120]:
df_ids['combined'] = df_ids[0].astype('str') + df_ids[1].astype('str') + df_ids[2].astype('str') + df_ids[3].astype('str')\
+ df_ids[4].astype('str') + df_ids[5].astype('str') + df_ids[6].astype('str') + df_ids[7].astype('str')
df_ids.drop(columns=[0,1,2,3,4,5,6,7],inplace=True)
df_ids.head()

,combined
0,_'s_hertogenboschNoneNoneNoneNoneNoneNone
1,_a_coruñaNoneNoneNoneNoneNoneNone
2,_aabenraaNoneNoneNoneNoneNoneNoneNone
3,_aasiaatNoneNoneNoneNoneNoneNoneNone
4,_abaNoneNoneNoneNoneNoneNoneNone


In [0]:
import re

In [0]:
def clean_pat(x):
  step1 = re.sub("None\,","",x)
  step2 = re.sub("None","",step1)
  step2 = '{"label": "GPE_city", "pattern": ' + '['+ step2.strip()[:-1] + ']'
  return step2


def clean_id(x):
  step1 = re.sub("None","",x)
  step2 = ', "id": "' + re.sub("(\_)([\'\_a-zA-Z\u0080-\uFFFF]+)",r"\2",step1) + '"}'
  return step2

In [0]:
df_pats['cleaned'] = df_pats['combined'].apply(lambda x: clean_pat(x))

df_pats.drop(columns=['combined'],inplace=True)
df_pats.columns=['cleaned_pattern']

df_ids['cleaned'] = df_ids['combined'].apply(lambda x: clean_id(x))
df_ids.drop(columns=['combined'],inplace=True)
df_ids.columns=['cleaned_id']


In [0]:
{"label": "vegetable", "pattern": [{"LOWER": "zuccchini"}], "id": "zucchini"}

In [124]:
df_full = pd.concat([df_pats,df_ids],axis=1)
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7602 entries, 0 to 7601
Data columns (total 2 columns):
cleaned_pattern    7602 non-null object
cleaned_id         7602 non-null object
dtypes: object(2)
memory usage: 118.9+ KB


In [125]:
df_full.head()

,cleaned_pattern,cleaned_id
0,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""'...",", ""id"": ""'s_hertogenbosch""}"
1,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""a_coruña""}"
2,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""aabenraa""}"
3,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""aasiaat""}"
4,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""aba""}"


In [126]:
df_full.tail()

,cleaned_pattern,cleaned_id
7597,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""ș...",", ""id"": ""ștefan_vodă""}"
7598,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""ʿ...",", ""id"": ""ʿamrān""}"
7599,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""ا...",", ""id"": ""الجلفة""}"
7600,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""م...",", ""id"": ""مأرب""}"
7601,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""ḍ...",", ""id"": ""ḍera_ismaīl_k͟hān""}"


In [0]:
df_full['combined'] = df_full['cleaned_pattern'].astype('str') + df_full['cleaned_id'].astype('str')

In [128]:
df_full.head()

,cleaned_pattern,cleaned_id,combined
0,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""'...",", ""id"": ""'s_hertogenbosch""}","{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""'..."
1,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""a_coruña""}","{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a..."
2,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""aabenraa""}","{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a..."
3,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""aasiaat""}","{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a..."
4,"{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a...",", ""id"": ""aba""}","{""label"": ""GPE_city"", ""pattern"": [{""LOWER"": ""a..."


In [130]:
df_full['combined'][0]

'{"label": "GPE_city", "pattern": [{"LOWER": "\'s"}, {"LOWER": "hertogenbosch"}], "id": "\'s_hertogenbosch"}'

In [132]:
type(list(df_full['combined']))

list

In [0]:
dump_jsonl(list(df_full['combined']), 'city_patterns.jsonl')

In [0]:
df_full[['combined']].to_csv('city_patterns.csv',index=False)

In [139]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10087 entries, 0 to 10086
Data columns (total 7 columns):
instance_of                                              10087 non-null object
instance_ofLabel                                         10087 non-null object
country                                                  9608 non-null object
countryLabel                                             9608 non-null object
located_in_the_administrative_territorial_entity         8466 non-null object
located_in_the_administrative_territorial_entityLabel    8466 non-null object
wikidata_entity_id                                       10087 non-null object
dtypes: object(7)
memory usage: 551.8+ KB


In [141]:
df_new = df2[['instance_ofLabel','located_in_the_administrative_territorial_entityLabel','countryLabel']]
df_new.columns = ['city','admin_division','country']
df_new.head()

,city,admin_division,country
0,Catania,Metropolitan City of Catania,Italy
1,Caserta,Province of Caserta,Italy
2,Juba,Central Equatoria,South Sudan
3,Dar es Salaam,Dar es Salaam Region,Tanzania
4,Khartoum,Khartoum,Sudan


In [143]:
for col in df_new.columns:
  df_new[col].apply(lambda row: row.str.lower())

AttributeError: ignored

In [0]:
def check_Qcodes(x):
  step1 = re.sub("Q[0-9]+","UNKNOWN",x)
  return step1

In [144]:
df_new['city'].apply(re.sub"Q[0-9]+","UNKNOWN",)

,city,admin_division,country
10082,Kerkrade,Kerkrade,Netherlands
10083,Q28976219,None,Jordan
10084,Q28976366,None,Jordan
10085,Q29000726,None,Jordan
10086,Al-Mashariqah,None,Jordan


In [159]:
df_new['city_check'] = df_new['city'].apply(lambda x: check_Qcodes(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [157]:
df_new.drop(labels='city_check',inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [160]:
df_new.tail()

,city,admin_division,country,city_check
10082,Kerkrade,Kerkrade,Netherlands,Kerkrade
10083,Q28976219,None,Jordan,UNKNOWN
10084,Q28976366,None,Jordan,UNKNOWN
10085,Q29000726,None,Jordan,UNKNOWN
10086,Al-Mashariqah,None,Jordan,Al-Mashariqah


In [163]:
df_new = df_new.query('city_check != "UNKNOWN"')
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8778 entries, 0 to 10086
Data columns (total 4 columns):
city              8778 non-null object
admin_division    8170 non-null object
country           8694 non-null object
city_check        8778 non-null object
dtypes: object(4)
memory usage: 342.9+ KB


In [0]:
df_new.drop(columns='city_check',inplace=True)

In [170]:
df_new.head()

,city,admin_division,country
0,catania,metropolitan city of catania,italy
1,caserta,province of caserta,italy
2,juba,central equatoria,south sudan
3,dar es salaam,dar es salaam region,tanzania
4,khartoum,khartoum,sudan


In [0]:
df_new['city'] = df_new['city'].str.lower()
df_new['admin_division'] = df_new['admin_division'].str.lower()
df_new['country'] = df_new['country'].str.lower()

In [172]:
df_new.query('country == "india"')

,city,admin_division,country
256,ahmedabad,gujarat,india
258,jamnagar,jamnagar taluka,india
260,mumbai,maharashtra,india
265,chennai,tamil nadu,india
267,bangalore,karnataka,india
...,...,...,...
9972,durgawati,patna,india
9977,"agar, madhya pradesh",agar malwa district,india
9978,nangal rajawatan,dausa district,india
9982,bandol seoni,None,india


In [0]:
check_admin = df_new[df_new['admin_division'].isna()].reset_index(drop=True)

In [191]:
df_new[df_new['admin_division'].notna()].reset_index(drop=True).groupby(['city'],as_index=False)['country'].count().sort_values(['country'],ascending=False)\


,city,country
1997,fort-de-france,11
5235,roubaix,9
4656,pau,8
458,avignon,8
1748,dunkirk,8
...,...,...
4864,portland,4
6667,warrnambool,4
6658,wangaratta,4
3750,martakert,4


In [195]:
df_new[df_new['admin_division'].notna()].reset_index(drop=True).query('city == "dunkirk"')

,city,admin_division,country
1677,dunkirk,nord,france
1678,dunkirk,canton of dunkerque-ouest,france
1679,dunkirk,canton of dunkerque-est,france
1680,dunkirk,arrondissement of dunkirk,france
1681,dunkirk,canton of grande-synthe,france
1682,dunkirk,canton of coudekerque-branche,france
1683,dunkirk,communauté urbaine de dunkerque,france
7531,dunkirk,matabeleland north province,zimbabwe


In [180]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8778 entries, 0 to 10086
Data columns (total 3 columns):
city              8778 non-null object
admin_division    8170 non-null object
country           8694 non-null object
dtypes: object(3)
memory usage: 594.3+ KB


In [181]:
df_new['admin_division'].value_counts()

são paulo                614
new south wales           23
victoria                  22
silesian voivodeship      18
minas gerais              16
                        ... 
urcos district             1
cazin municipality         1
yamuna nagar district      1
orkdal                     1
cimișlia district          1
Name: admin_division, Length: 5439, dtype: int64